In [ ]:
from cling import cling, bash

**Listing 7.1**

Caption: Examples for a race condition.

In [ ]:
%%writefile race_condition.cpp
#include <thread>
#include <iostream>
//-----------------------------------------------------------------
double sum = 0;
const int sz = 1000000;
double* data = new double[sz];

void update(int n) {
    for(int i=0;i<sz;i++)
        sum += data[i];
}

int main() {
    for(int i=0;i<sz;i++) data[i] = 1.0;
    std::thread t1(update,1), t2(update,2);
    t1.join(); t2.join();
    std::cout << "final sum=" << sum << std::endl;
    return 0;
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o race_condition.exe race_condition.cpp -lpthread
./race_condition.exe

**Listing 7.2**

Caption: Another example for a race condition.

In [ ]:
%%writefile race_condition2.cpp
#include <thread>
#include <iostream>
//-----------------------------------------------------------------
double sum = 0;
const int sz = 40;
double* data = new double[sz];

void update(int n) {
    for(int i=0;i<sz;i++) {
        sum += data[i];
        std::cout << "working sum=" << sum << " at step=" << i << std::endl;
    }
}

int main() {
    for(int i=0;i<sz;i++) data[i] = 1.0;
    std::thread t1(update,1), t2(update,2);
    t1.join(); t2.join();
    std::cout << "final sum=" << sum << std::endl;
    return 0;
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o race_condition2.exe race_condition2.cpp -lpthread
./race_condition2.exe

**Listing 7.3**

Caption: Usage of a mutex to avoid the race condition, but execute the code sequential

In [ ]:
%%writefile mutex.cpp
#include <thread>
#include <iostream>
#include <mutex>
//-----------------------------------------------------------------
double sum = 0;
const int sz = 1000000;
double* data = new double[sz];

std::mutex m;

void update(int n) {
    for(int i=0;i<sz;i++) {
        m.lock();
        sum += data[i];
        m.unlock();
    }
}

int main() {
    for(int i=0;i<sz;i++) data[i] = 1.0;
    std::thread t1(update,1), t2(update,2);
    t1.join(); t2.join();
    std::cout << "final sum=" << sum << std::endl;
    return 0;
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o mutex.exe mutex.cpp -lpthread
./mutex.exe

**Listing 7.4**

Caption: Improved version of the code using a mutex to avoid the race condition.

In [ ]:
%%writefile mutex2.cpp
#include <thread>
#include <iostream>
#include <mutex>
//-----------------------------------------------------------------
double sum = 0;
const int partial_sz = 1000;
const int sz = partial_sz*partial_sz;
double* data = new double[sz];

std::mutex m;

void update(int n) {
    for(int j=0;j<partial_sz;j++) {
        // Do part of the sum on a local variable, partial_sum
        double partial_sum = 0;
        for(int i=0;i<partial_sz;i++) {
            partial_sum += data[i];
        }
        // update the result using the lock
        m.lock();
        sum += partial_sum;
        m.unlock();
    }
}

int main() {
    for(int i=0;i<sz;i++) data[i] = 1.0;
    std::thread t1(update,1), t2(update,2);
    t1.join(); t2.join();
    std::cout << "final sum=" << sum << std::endl;
    return 0;
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o mutex2.exe mutex2.cpp -lpthread
./mutex2.exe

**Listing 7.5**

Caption: Example for a deadlock where two threads block each other and the program will never terminate.

In [ ]:
%%writefile deadlock.cpp
#include <thread>
#include <mutex>
#include <iostream>
//-----------------------------------------------------------------
int main() {
    std::mutex locka, lockb;
    std::thread ta([&](){
        locka.lock();
        std::cout << "Got lock A" << std::endl;
        lockb.lock();
        std::cout << "Got lock A then B" << std::endl;
        lockb.unlock();
        locka.unlock();
    });
    std::thread tb([&](){
        lockb.lock();
        std::cout << "Got lock B" << std::endl;
        locka.lock();
        std::cout << "Got lock B then A" << std::endl;
        locka.unlock();
        lockb.unlock();
    });
    ta.join();
    tb.join();
    return 0;
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o deadlock.exe deadlock.cpp -lpthread
./deadlock.exe

**Listing 7.6**

Caption: Attempt to avoid the deadlock using a scope\_lock.

In [ ]:
%%writefile scoped_lock.cpp
#include <thread>
#include <mutex>
#include <iostream>
//-----------------------------------------------------------------
int main() {
    std::mutex locka, lockb;
    std::thread ta([&](){
        std::scoped_lock slock{locka, lockb};
        std::cout << "Got lock A then B" << std::endl;
    });
    std::thread tb([&](){
        std::scoped_lock slock{lockb, locka};
        std::cout << "Got lock B then A" << std::endl;
    });
    ta.join();
    tb.join();
    return 0;
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o scoped_lock.exe scoped_lock.cpp -lpthread
./scoped_lock.exe

**Listing 7.7**

Caption: Computation of the partial sum using std::atomic.

In [ ]:
%%writefile atomic.cpp
#include <thread>
#include <iostream>
#include <atomic>
//-----------------------------------------------------------------
std::atomic<double> sum = 0;
const int partial_sz = 1000;
const int sz = partial_sz*partial_sz;
double* data = new double[sz];

void update(int n) {
    for(int j=0;j<partial_sz;j++) {
        // Do part of the sum on a local variable, partial_sum
        double partial_sum = 0;
        for(int i=0;i<partial_sz;i++) {
            partial_sum += data[i];
        }
        while(true) {
            double expected = sum.load();
            double desired = expected + partial_sum;
            // try update, if success break
            if(sum.compare_exchange_strong(expected, desired))
                break;
        }
    }
}

int main() {
    for(int i=0;i<sz;i++) data[i] = 1.0;
    std::thread t1(update,1), t2(update,2);
    t1.join(); t2.join();
    std::cout << "final sum=" << sum << std::endl;
    return 0;
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o atomic.exe atomic.cpp -lpthread
./atomic.exe